In [1]:
from pyspark import SparkConf ,SparkContext
from pyspark.sql.session import SparkSession


conf = SparkConf().setMaster("local").setAppName("word-counts")

sc = SparkContext(conf=conf)

In [110]:
d = {}
for i in range(len(os.listdir("dataset"))):
    bookRawRDD = sc.textFile("dataset/" + str(i))
    bookRawRDD.collect()[:10]
    bookMessyTokensRDD = bookRawRDD.flatMap(lambda line: line.lower().strip().split(" "))
    bookCleanTokensRDD = bookMessyTokensRDD.map(lambda letter: re.sub(r'[^A-Za-z]', '', letter))
    
    #remove stop words
    from pyspark.ml.feature import StopWordsRemover
    remover = StopWordsRemover()
    stopwords = remover.getStopWords()
    bookwordsRDD = bookCleanTokensRDD.filter(lambda PointLessW: PointLessW not in stopwords)
    
    bookEmptyRemoveRDD = bookwordsRDD.filter(lambda x: x != "")
    
    bookPairsRDD = bookEmptyRemoveRDD.map(lambda word: (word,1))
    #print(bookPairsRDD.collect()[:1])
    
    bookWordCountRDD = bookPairsRDD.reduceByKey(lambda acc, value:acc +value )
    
    bookResults = bookWordCountRDD.map(lambda x: (x[0])).collect()
    
    import pandas as pd
    df = pd.DataFrame(bookResults)
    df.insert(1, "ID", range(0, len(df)))
    d[i] = bookResults
    #list = df[i].values.tolist()
    #print(bookResults
    #sparkDF=sc.createDataFrame(df) 
    #sparkDF.printSchema()
    #sparkDF.show()

print(d[6])

['project', 'gutenberg', 'etext', 'eight', 'years', 'wanderings', 'ceylon', 'samuel', 'white', 'baker', 'copyright', 'laws', 'changing', 'world', 'sure', 'check', 'country', 'posting', 'files', 'please', 'take', 'look', 'important', 'information', 'header', 'encourage', 'keep', 'file', 'disk', 'keeping', 'electronic', 'path', 'open', 'next', 'readers', 'remove', 'welcome', 'free', 'plain', 'vanilla', 'texts', 'etexts', 'readable', 'humans', 'computers', 'since', 'prepared', 'hundreds', 'volunteers', 'donations', 'contacting', 'get', 'included', 'need', 'january', 'named', 'yeartxt', 'yearzip', 'corrected', 'editions', 'new', 'number', 'versions', 'based', 'separate', 'sources', 'letter', 'yearatxt', 'garry', 'gill', 'garrygillhotmailcom', 'usually', 'created', 'multiple', 'public', 'domain', 'united', 'states', 'unless', 'notice', 'therefore', 'books', 'compliance', 'particular', 'paper', 'edition', 'trying', 'release', 'one', 'month', 'advance', 'official', 'dates', 'leaving', 'time',

In [104]:
print(type(d))

<class 'dict'>


In [5]:

bookRawRDD = sc.textFile("dataset/1")
bookRawRDD.collect()[:10]

['Project Gutenberg Etext Critical & Historical Essays, by Macaulay',
 '#7 in our series by Thomas Babington Macaulay',
 '',
 '',
 'Copyright laws are changing all over the world, be sure to check',
 'the copyright laws for your country before posting these files!!',
 '',
 'Please take a look at the important information in this header.',
 'We encourage you to keep this file on your own disk, keeping an',
 'electronic path open for the next readers.  Do not remove this.']

In [6]:
bookMessyTokensRDD = bookRawRDD.flatMap(lambda line: line.lower().strip().split(" "))

In [7]:
import re
bookCleanTokensRDD = bookMessyTokensRDD.map(lambda letter: re.sub(r'[^A-Za-z]', '', letter))

In [8]:
#remove stop words
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover()
stopwords = remover.getStopWords()
bookwordsRDD = bookCleanTokensRDD.filter(lambda PointLessW: PointLessW not in stopwords)

In [9]:
bookEmptyRemoveRDD = bookwordsRDD.filter(lambda x: x != "")

In [67]:
bookPairsRDD = bookEmptyRemoveRDD.map(lambda word: (word,1))
bookPairsRDD.collect()[:10]

[('project', 1),
 ('gutenberg', 1),
 ('etext', 1),
 ('way', 1),
 ('flesh', 1),
 ('samuel', 1),
 ('butler', 1),
 ('series', 1),
 ('samuel', 1),
 ('butler', 1)]

In [68]:
i = 0
bookWordCountRDD = bookPairsRDD.reduceByKey(lambda acc, value: acc +value )

In [74]:
i=0
bookResults = bookWordCountRDD.map(lambda x: (x[0])).sortByKey(False).take(10)
bookResults = bookWordCountRDD.map(lambda x: (x[0])).take(10)
print(type(bookResults))

<class 'list'>
